# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,faya,18.3851,42.4509,22.06,35,21,2.60,SA,1666108228
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
3,3,jamestown,42.0970,-79.2353,5.77,77,100,9.77,US,1666107934
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    size = 'Humidity'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df['Max Temp'] <= 22) & (city_data_df['Max Temp'] >= 10)]

# Drop any rows with null values

ideal_weather = ideal_weather.dropna()

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,farsund,58.0948,6.8047,13.30,100,0,7.65,NO,1666108228
2,2,new norfolk,-42.7826,147.0587,11.72,58,12,1.34,AU,1666108230
4,4,lanzhou,36.0564,103.7922,14.53,48,59,1.20,CN,1666108230
6,6,albany,42.6001,-73.9662,10.36,74,29,2.10,US,1666108231
7,7,carnarvon,-24.8667,113.6333,20.02,76,0,7.46,AU,1666108232
...,...,...,...,...,...,...,...,...,...,...
559,559,lompoc,34.6391,-120.4579,17.55,100,0,3.60,US,1666108556
564,564,redmond,47.6740,-122.1215,13.17,91,8,0.17,US,1666108556
565,565,wyndham,37.6985,-77.6125,12.81,52,3,1.79,US,1666108480
571,571,dargaville,-35.9333,173.8833,10.97,83,13,3.83,NZ,1666108563


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = pd.DataFrame(city_data_df[['City', 'Lat', 'Lng', 'Humidity']])


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(**{'Hotel Name': ''})

# Display sample data
hotel_df

,City,Lat,Lng,Humidity,Hotel Name
0,faya,18.3851,42.4509,35,
1,farsund,58.0948,6.8047,100,
2,new norfolk,-42.7826,147.0587,58,
3,jamestown,42.0970,-79.2353,77,
4,lanzhou,36.0564,103.7922,48,
...,...,...,...,...,...
572,sovetskiy,61.3614,63.5842,96,
573,tatarsk,55.2190,75.9828,89,
574,canchungo,12.0672,-16.0333,61,
575,iquique,-20.2208,-70.1431,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
 # Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
filters = []
bias = []
limit = 1

params = {
    "categories":categories,
    "filter":filters,
    "bias":bias,
    "limit":limit,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
faya - nearest hotel: No hotel found
farsund - nearest hotel: Rederiet Hotell
new norfolk - nearest hotel: Woodbridge on the Derwent
jamestown - nearest hotel: DoubleTree Jamestown
lanzhou - nearest hotel: 西园宾馆
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
albany - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
sisimiut - nearest hotel: Sømandshjemmet
saint-pierre - nearest hotel: Tropic Hotel
colorado - nearest hotel: Servilha Park Hotel
nikolskoye - nearest hotel: No hotel found
trairi - nearest hotel: Pousada Villa Aurora
dobryanka - nearest hotel: Уральская Венеция
qaanaaq - nearest hotel: Hotel Qaanaaq
mataura - nearest hotel: No hotel found
khatanga - nearest hotel: Хатанга
tasiilaq - nearest hotel: Angmagssalik
tiksi - nearest hotel: Арктика
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
nuevo laredo - nearest hotel: Best Western Plus
guiratinga - nearest hotel: No hot

cacahuatepec - nearest hotel: No hotel found
paita - nearest hotel: Victor's
bumba - nearest hotel: No hotel found
maragogi - nearest hotel: Pousada Mariluz
kosa - nearest hotel: Гостиница "Рады"
lagoa - nearest hotel: Casa da Madrinha
cabra - nearest hotel: Pensión Guerrero
gat - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
asau - nearest hotel: Hotel Romanița
isangel - nearest hotel: Tanna Lodge
pevek - nearest hotel: No hotel found
husavik - nearest hotel: Fosshótel Húsavík
kodiak - nearest hotel: Shelikof Lodge
zyryanka - nearest hotel: No hotel found
valparaiso - nearest hotel: Ibis Hotel
baykit - nearest hotel: No hotel found
port moresby - nearest hotel: The Sanctuary Hotel and Spa
meulaboh - nearest hotel: Hotel Meuligou
nakamura - nearest hotel: ホテルココモ
simao - nearest hotel: Marvelous Land Hotel
laguna - nearest hotel: Holiday Inn Express & Suites
bethel - nearest hotel: Hampton Inn Danbury
kodinsk - nearest hotel: Ангара
luderitz - nearest hotel: Seavie

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    hover_cols=['City', 'Country', 'Hotel Name'],
    size=10,
    alpha=0.7,
    geo = True,
    tiles = 'OSM'
)

# Display the map
map_plot